### Visualizing robot senses using ROS and Bokeh library

As you can see, robots publish many topics that could be interesting to see. 
Visualizing state of a sensor is a first step to programming using this sense or debugging if something goes wrong.

We will start with visualizing the state of the turtlesim through some cool techniques.

In [1]:
%env ROS_MASTER_URI=http://localhost:11311
import rospy
import roslaunch
import rosnode



env: ROS_MASTER_URI=http://localhost:11311


In [2]:
import os

os.system("rosrun ipython_robot_prototyping roomba_simulator.py &")
os.system("rosrun turtlesim turtlesim_node &")

0

With the fallowing commands we have started two ROS Nodes -- one is a familiar turtlebot simulator and the other is a simple control node for the turtle that turns turtle around when she is near the wall so that it does not crash.


What is also does is it calculates the distance of the turtle to the wall she is heading to.

Lets see all the turtle's topics

In [14]:
turtle_topics= tuple((topic_name,topic_type) 
                     for topic_name, topic_type in rospy.get_published_topics()
                     if "turtle" in topic_name)

print(turtle_topics)

(('/turtle1/distance', 'sensor_msgs/Range'), ('/turtle1/color_sensor', 'turtlesim/Color'), ('/turtle1/cmd_vel', 'geometry_msgs/Twist'), ('/turtle1/pose', 'turtlesim/Pose'))


There are couple of interesting topics there, like distance or pose. How we can visualise them?

In case of turtlesim it is easy to see what the distance or position are by just looking at it. That is nice but in case of real robots we are not only interested in the actual position and orientation but also on robots _sensor_ readings that is, robots sensors. The distance in /turtle1/distance is something calculated, but does it actally make sense? It is very important to have a way to visualise sensor data and luckyly, there are many ways in ROS to do it:

 - The easiest way is to use _rostopic echo_ command. You just put, for example rostopic echo /turtle1/distance to see the distance values. Try to do it now.
 
 - The second option is to use another ROS command -- rqt_plot.


In [19]:
os.system("rqt_plot /turtle1/distance/range")

0

The command is nice as it automaticly scales a plot to fit the changing data. It also allows you to add different topics through graphical user interface or by putting different topic names when starting the tool( i.e. rqt_prot /turtle1/distance/range /turtle1/pose/x ). There is also a settings button in the right top corner to further set things up

 - It would be nice though to see the things directely in the Ipython notebook, to be able to see how the things change when we play around. We will show you couple of ways to do that

### Ipython widgets

The first way, especially if there is a single changing value is to use IPython slider but as an output view.

We will to this by using a _traitlet_ that is an element that, when changed, causes change in slider.



In [26]:
from chatterbot import ChatBot
chatbot = ChatBot("Ron Obvious")

from chatterbot.trainers import ListTrainer



from gtts import gTTS


conversation = [
    "Hello",
    "Hi there!",
    "How are you doing?",
    "I'm doing great.",
    "That is good to hear",
    "Thank you.",
    "You're welcome.",
    "My name is Igor",
    "and my name is Marcin",
    "I am 8 years old",
    "and I am a pedophile",
    "I am 18 years old",
    "I am not interested"
]


chatbot.set_trainer(ListTrainer)
chatbot.train(conversation)



import os

def gadaj(tekst):
    tts = gTTS(text=str(tekst), lang='en', slow=True)
    tts.save("gadaj.mp3")
    os.system("mpg123 gadaj.mp3")
 



In [35]:
response = chatbot.get_response("You are retard")
gadaj(response)

In [ ]:
tts.

In [11]:
print(chatbot.get_response("My name is Kutas"))

and my name is Marcin
